To test that linear response theory is recovering the true amplification of freshwater fluxes, we need to evaluate the amplification of fluxes from CESM flux fields. Note that as linear response theory is finding the change in freshwater fluxes as a proportion of the FAFMIP perturbation, the truth is not exactly the change in magnitude of freshwater fluxes. Here we show the test with the integration metric defined in the paper

In [10]:
import scipy.io
import netCDF4
import xarray as xr
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import os, glob 
import imageio
from matplotlib import animation
import copy
import cartopy as cart
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter #see https://scitools.org.uk/cartopy/docs/v0.15/examples/tick_labels.html
import certifi
import ssl
import math
ssl._create_default_https_context = ssl._create_unverified_context
from scipy import stats
from xgcm import Grid
import statsmodels.api as sm
import matplotlib.ticker as ticker
from matplotlib.axes._secondary_axes import SecondaryAxis
import xesmf as xe

In [11]:
#First, load in all the freshwater fluxes. See notebook processing_salt_fluxes.py for how to generate this files that we unpickle here
import pickle 

with open("/scratch/abf376/regridded_salt_flux_2005on", "rb") as fp:   #Unpickling
    regridded_2005on=pickle.load(fp)

## First we calculate the true fluxes for the ensemble mean using the integration metric

In [3]:
#average over ensemble member
salt_flux_avg_2005on=sum(regridded_2005on)/34
salt_flux_avg_2005on=salt_flux_avg_2005on.rename({'y': 'latitude','x': 'longitude'})
salt_flux_avg_2005on=salt_flux_avg_2005on.assign_coords(latitude=salt_flux_avg_2005on.lat[:,0],longitude=salt_flux_avg_2005on.lon[0,:])

In [4]:
salt_flux_avg_2006to2055=salt_flux_avg_2005on[0:12*59,:,:] #salt_avg was from 1920 to 2080 so this is from 1970 to 2019

In [5]:
#create area grid

import sys
sys.path.insert(1, '/scratch/abf376/')
from area_grid import *

area=area_grid(latitudes=np.array(salt_flux_avg_2005on[0,:,:].latitude),longitudes=salt_flux_avg_2005on[0,:,:].longitude)
area=xr.DataArray(area,dims=["latitude","longitude"],coords=[salt_flux_avg_2005on[0,:,:].latitude,salt_flux_avg_2005on[0,:,:].longitude])

In [6]:
#water flux from fafmip
f='/scratch/abf376/FAFMIP_wfo_v2.nc' #this is the first 50 years
file2read = netCDF4.Dataset(f,'r')
#print(file2read.variables)
wfo = xr.open_dataset(f)['water_flux_into_sea_water']
wfo=wfo.where(wfo<1E19)


area_wfo=area_grid(latitudes=np.array(wfo.latitude),longitudes=wfo.longitude)
area_wfo=xr.DataArray(area,dims=["latitude","longitude"],coords=[wfo.latitude,wfo.longitude])

In [7]:
#regrid wfo
ds_out = xe.util.grid_global(1, 1)
regridder_wfo= xe.Regridder(wfo, ds_out, "bilinear",periodic=True)
wfo = regridder_wfo(wfo)
wfo=wfo.rename({'y': 'latitude','x': 'longitude'})
wfo=wfo.assign_coords(latitude=wfo.lat[:,0],longitude=wfo.lon[0,:])

/ext3/miniconda3/lib/python3.9/site-packages/xarray/core/dataarray.py:780: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return key in self.data


In [8]:
EP_pattern_wfo=np.empty(50)
for i in range(0,50):
    EP_pattern_wfo[i]=np.abs((((salt_flux_avg_2006to2055[i*12:(i+1)*12,:,:].mean('time')).where(np.sign(wfo.mean('time'))>0))*area.where(area.latitude<65)).sum())+np.abs((((salt_flux_avg_2006to2055[i*12:(i+1)*12,:,:].mean('time')).where(np.sign(wfo.mean('time'))<0))*area.where(area.latitude<65)).sum())

In [9]:
np.random.seed(0)
A=((np.abs(wfo.mean('time')))*area.where(area.latitude<65)).sum() #strength of wfo in this way, 9.99660706e+08 if calculated with regridded or 1.08156723e+09 if not calculated with regridded


change2=np.empty([3000])
trend=np.empty([45])
p=scipy.stats.linregress(np.linspace(0,44,45), y=EP_pattern_wfo[5:50], alternative='two-sided')
trend=p.intercept+p.slope*np.linspace(0,44,45)

from recombinator.block_bootstrap import circular_block_bootstrap

# number of replications for bootstraps (number of resampled time-series to generate)
B = 3000

y_star_cb \
    = circular_block_bootstrap(EP_pattern_wfo[5:50]-trend, 
                               block_length=2, 
                               replications=B, replace=True)
bootstrap=[]
for i in range(0,B):
    bootstrap.append(trend+y_star_cb[i,:])

for i in range(0,B):
    change2[i]=(bootstrap[i][40:45].mean()-bootstrap[i][0:5].mean())/A

print(change2.mean())
print(change2.std())

7.338997043114277e-01
0.04939843236937896


## Now, we need to do the same for each individual member

In [12]:
# a list of all ensemble members
for i in range(0,34):
    regridded_2005on[i]=regridded_2005on[i].rename({'y': 'latitude','x': 'longitude'})
    regridded_2005on[i]=regridded_2005on[i].assign_coords(latitude=salt_flux_avg_2005on.latitude,longitude=salt_flux_avg_2005on.longitude)
    
salt_avg_list=[]
for i in range(0,34):
    salt_avg_list.append(regridded_2005on[i])
    
climatological_salt_flux_list=[]
for i in range(0,34):
    climatological_salt_flux_list.append(salt_avg_list[i][0:12*50,:,:].mean('time')) #this is 1920 to 1975
    salt_avg_list[i]=salt_avg_list[i][0:12*50,:,:]

In [13]:
EP_pattern_wfo_list=np.empty([34,50])
for j in range(0,34):
    for i in range(0,50):
        EP_pattern_wfo_list[j,i]=np.abs((((salt_avg_list[j][i*12:(i+1)*12,:,:].mean('time')).where(np.sign(wfo.mean('time'))>0))*area.where(area.latitude<65)).sum())+np.abs((((salt_avg_list[j][i*12:(i+1)*12,:,:].mean('time')).where(np.sign(wfo.mean('time'))<0))*area.where(area.latitude<65)).sum())

In [14]:
np.random.seed(0)
A=((np.abs(wfo.mean('time')))*area.where(area.latitude<65)).sum() #strength of wfo in this way, 9.99660706e+08 if calculated with regridded or 1.08156723e+09 if not calculated with regridded

change=np.empty([3000,34])
for j in range(0,34):
    trend=np.empty([45])
    p=scipy.stats.linregress(np.linspace(0,44,45), y=EP_pattern_wfo_list[j,5:50], alternative='two-sided')
    trend=p.intercept+p.slope*np.linspace(0,44,45)

    from recombinator.block_bootstrap import circular_block_bootstrap

    # number of replications for bootstraps (number of resampled time-series to generate)
    B = 3000

    y_star_cb \
        = circular_block_bootstrap(EP_pattern_wfo_list[j,5:50]-trend, 
                                   block_length=2, 
                                   replications=B, replace=True)
    bootstrap=[]
    for i in range(0,B):
        bootstrap.append(trend+y_star_cb[i,:])

    for i in range(0,B):
        change[i,j]=(bootstrap[i][40:45].mean()-bootstrap[i][0:5].mean())/A

mean_boot_m2=np.empty([34])
std_boot_m2=np.empty([34])
for j in range(0,34):
    mean_boot_m2[j]=change[:,j].mean()
    std_boot_m2[j]=change[:,j].std()

In [16]:
import pickle
with open("freshwater_fluxes_integration_mean_boot_2011to2055_new", "wb") as fp:   #Pickling
    pickle.dump(mean_boot_m2, fp)
with open("freshwater_fluxes_integration_std_boot_2011to2055_new", "wb") as fp:   #Pickling
    pickle.dump(std_boot_m2, fp)

ensemble_projection_mean_std=[change2.mean(),change2.std()]
with open("freshwater_fluxes_ensemble_integration_mean_std_2011to2055_new", "wb") as fp:   #Pickling
    pickle.dump(ensemble_projection_mean_std, fp)